In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr


# import https
import urllib.request
import json


# Basic Dataset erstellen

In [ ]:
df_rest = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
df_act = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
df_hotel = pd.read_csv('01_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')

print('Restaurants (Shape): ', df_rest.shape)
print('Activities (Shape): ', df_act.shape)
print('Hotels (Shape): ', df_hotel.shape)

In [85]:
columns_to_keep = [
    "stars",
    "publishedAtDate",
    "name",
    "text",
    "title",
    "totalScore",
    "likesCount",
    "isLocalGuide",
    "reviewId",
    "categoryName",
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
    "reviewImageUrls/6",
    "reviewImageUrls/7",
    "reviewImageUrls/8",
    "reviewImageUrls/9",
    "reviewImageUrls/10",
    "reviewUrl",
    "reviewerId",
    "reviewerNumberOfReviews",
    "reviewerPhotoUrl",
    "reviewerUrl",
    "reviewsCount",
    "scrapedAt",
    "state",
    "street",
    "temporarilyClosed",
    "url",
    "website"
]

In [14]:
df_rest = df_rest.dropna(subset=["reviewImageUrls/0"])
df_rest = df_rest[columns_to_keep]
print(df_rest.shape)

(1179, 33)


In [15]:
df_act = df_act.dropna(subset=["reviewImageUrls/0"])
df_act = df_act[columns_to_keep]
print(df_act.shape)

(3844, 33)


In [16]:
df_hotel = df_hotel.dropna(subset=["reviewImageUrls/0"])
df_hotel = df_hotel[columns_to_keep]
print(df_hotel.shape)

(1846, 33)


In [20]:
df = pd.concat([df_rest, df_act, df_hotel], ignore_index=True)
print(df.shape)

(6869, 33)


## Gradio Anwendung Versuch

In [94]:
def generate_review_html(stars, date, user, local_guide, text):
    html = f"""
    <div class=\"review\">
      <div class=\"user\">
        <img src=\"https://via.placeholder.com/50\" alt=\"User profile picture\">
        <div class=\"user-info\">
          <h4>{user}</h4>
            <p>Submitted on {date}</p>
        </div>
        </div>
        <div class=\"rating\">
        <span class=\"star\">&#9733;</span>
        <span class=\"star\">&#9733;</span>
        <span class=\"star\">&#9733;</span>
        <span class=\"star\">&#9733;</span>
        <span class=\"star\">&#9733;</span>
        </div>
        <div class=\"review-text\">
        <p>{text}</p>
        </div>
        <div class=\"review-image\">
        <img src=\"http://via.placeholder.com/150/%22%20alt=/%22Restaurant%20image/\" alt=\"Restaurant image\">
        </div>
    </div>
    """
    return html

def generate_deepfake(review_about):
    text = review_about + " is a great place to visit."
    meta_info = pd.DataFrame(columns=["stars", "date", "user", "local_guide"])
    meta_info.loc[0] = [4, "2021-01-01", "John Doe", False]
    stars = 1
    date = "2021-01-01"
    user = "John Doe"
    local_guide = False
    text = "This is a great place to visit. I will definitly come back. Love it"

    html_review = generate_review_html(stars, date, user, local_guide, text)

    return html_review

def verify_review(stars, date, user, local_guide, text, image):
    return "FAKE", "0.9"

with gr.Blocks() as demo:
    with gr.Tab ("Create Fake Review"):
        review_about = gr.Textbox(label="Review about")
        generate_btn = gr.Button("Generate DeepFake")
        # output_image = gr.Textbox(label="Review Image")
        # output_text = gr.Textbox(label="Generated Text")
        # output_tabular = gr.Dataframe(label="Meta Information")

        html_output = gr.HTML(label="Review")
        generate_btn.click(fn=generate_deepfake, inputs=review_about, outputs=[html_output])
    
    with gr.Tab ("Verify Review"):
        with gr.Row(): 
            input_stars = gr.Slider(label="Stars", minimum=1, maximum=5, step=1)
            input_date = gr.Textbox(label="Date", default="2023-01-01")
            input_user = gr.Textbox(label="User")
            input_local_guide = gr.Checkbox(label="Local Guide")
       
        input_text = gr.Textbox(label="Review Text")
        input_image = gr.Image(label="Review Image")
        verify_btn = gr.Button("Verify Review")
        output_text = gr.Textbox(label="Generated Text")
        output_score = gr.Textbox(label="Score")
        verify_btn.click(fn=verify_review, inputs=[input_stars, input_date, input_user, input_local_guide, input_text, input_image], outputs=[output_text, output_score])

demo.launch(debug=True)

c:\Users\michi\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'default': '2023-01-01'}
  warnings.warn(


Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
